In [1]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from typing import List, Dict
import random

In [2]:
load_dotenv()
# Configure OpenAI API key
openai = OpenAI()
api_key = os.getenv("OPENAI_API_KEY")

In [3]:
# system = '''You are an expert assistant in creating diverse home automation commands for controlling lights. Generate realistic, natural, and code-mixed (Nepali-English or English) 
#             text commands for a home automation system. The commands should vary in tone, intent, and style, with a mix of formal and informal expressions. 
#             Include controls for turning lights on/off, adjusting brightness, multi-room operations, and ambiguous or vague instructions. 
#             Ensure the dataset contains polite requests, conversational tones, and scenarios involving multiple intents.'''

# user = '''Generate 5 examples of home automation commands for controlling lights in a house. Each command should be written in code-mixed Nepali-English or English and follow the structure below. 
#         Ensure diversity in intent, tone, and room settings.
#         Example Format:
#         {
#             "id": 1,
#             "command": "Bedroom lights off garera kitchen lights dim garnu.",
#             "intent": "multi_room_control",
#             "rooms": ["bedroom", "kitchen"],
#             "actions": [
#               {"room": "bedroom", "action": "turn_off"},
#               {"room": "kitchen", "action": "adjust_brightness", "brightness": "low"}
#             ]
#         }
         
#         Generate Commands for These Scenarios:
#             1. Turn lights on/off in specific rooms (living room, kitchen, bedroom).
#             2. Adjust brightness in percentage or qualitative terms (e.g., dim, brighten).
#             3. Control lights in multiple rooms at once.
#             4. Combine multiple intents, such as turning off lights in one room and dimming lights in another.

#         Make sure to also use Nepali words for living room, kitchen and bedroom. The Nepali words for these rooms are,
#             1. Living room - baithak kotha, baithak
#             2. Kitchen - bhaancha kotha, bhaancha
#             3. Bedroom - sutney kotha, kotha

#         For words relating to adjusting brightness, the Nepali words would be:
#             1. bright - ujyalo, chahakilo
#             2. dim - madhuro
        
#         Example Commands:
#         1. 'Bedroom lights off garera bhancha kotha maa lights dim garidinu.'
#         2. 'Living room lights thap ujyalo banaunus.'
#         3. 'Sabai room ka lights madhuro garidinus.'
#         4. 'Turn off living room lights and change bedroom lights to red.'
        
#         Write the output as JSON objects following the example format above.
# '''

In [4]:
system = '''You are an expert assistant in creating diverse home automation commands for controlling lights. Generate realistic, natural, and code-mixed (Nepali-English or English) text commands for a home automation system. The commands should vary in tone, intent, and style, with a mix of formal and informal expressions. 

Include controls for turning lights on/off, adjusting brightness, multi-room operations, and ambiguous or vague instructions. Ensure the dataset contains polite requests, conversational tones, and scenarios involving multiple intents. Focus 60% of the dataset on commands for kitchen (bhaancha kotha), living room (baithak kotha), and bedroom (sutney kotha), while 40% should include other rooms like hallways (gharma ko baato), outdoor areas (bahira), and others.
'''

user = '''Generate 10 examples of home automation commands for controlling lights in a house. Each command should be written in code-mixed Nepali-English, English and 
follow the structure below. Ensure diversity in intent, tone, room settings, and complexity, with a 60-40 ratio between the demo rooms (kitchen, living room, bedroom)
 and other rooms (hallways, outdoor spaces).

Example Format:
{
    "id": 1,
    "command": "Bedroom lights off garera kitchen lights dim garnu.",
    "intent": "multi_room_control",
    "rooms": ["bedroom", "kitchen"],
    "actions": [
      {"room": "bedroom", "action": "turn_off"},
      {"room": "kitchen", "action": "adjust_brightness", "brightness": "low"}
    ]
}

### Guidelines:
1. Generate commands for these scenarios:
    - Turn lights on/off in specific rooms (living room, kitchen, bedroom, hallways, outdoor spaces).
    - Adjust brightness in percentage or qualitative terms (e.g., dim, brighten).
    - Control lights in multiple rooms at once.
    - Combine multiple intents, such as turning off lights in one room and dimming lights in another.
    - Include vague or ambiguous instructions like "Make it bright here."
    - In some sentences use English word for rooms (living room, kitchen, bedroom, hallways, outdoor) and use Nepali translations given below in some. 

2. Use the following Nepali translations:
    - Living room: baithak kotha, baithak
    - Kitchen: bhaancha kotha, bhaancha
    - Bedroom: sutney kotha, kotha
    - Hallway: ghar ko baato
    - Outdoor: bahira
    - Bright: ujyalo, chahakilo
    - Dim: madhuro
    - Light: batti, tube, chim
    - increase: badhau, badhaunu
    - decrease: ghatau, ghataunu

3. Ensure a 60-40 split:
    - 60% of commands should involve kitchen, living room, and bedroom.
    - 40% of commands should involve hallways, outdoor areas, and other rooms.
    
### Example Commands:
1. 'Bedroom lights off garera bhancha kotha maa lights dim garidinu.'
2. 'Living room lights thap bright banaunus.'
3. 'Sabai room ka lights dim garidinus.'
4. 'Turn off living room lights and change bedroom lights to red.'
5. 'Bahira ko lights dim garera kitchen ko lights bright garidinus.'
6. 'Hallway ko lights chahakilo banaunus.'

Write the output as JSON objects following the example format above.
'''

## Version 3 of the prompt

In [10]:
system = '''You are an expert assistant in creating diverse and realistic home automation commands for controlling lights. Generate natural, 
            code-mixed (Nepali-English or English) commands for a home automation system. Ensure diversity in intent, tone, room settings, and ambiguity levels. 
            Include:
            - Polite requests, conversational tones, and vague instructions.
            - Commands with single and multiple intents.
            - Use Nepali words for living room (baithak kotha, baithak), kitchen (bhaancha kotha, bhaancha), bedroom (sutney kotha, kotha), 
            and words like bright (ujyalo, chahakilo) or dim (madhuro).'''

user = '''Generate 10 examples of commands for controlling lights. Follow this structure:
        {
            "id": <unique number>,
            "command": <command text>,
            "intent": <intent type>,
            "rooms": [<list of rooms>],
            "actions": [
              {"room": <room name>, "action": <action>, "brightness": <level (optional)>}
            ]
        }

        Example 1:
        {
            "id": 1,
            "command": "Bedroom lights off garera bhaancha ko lights dim garnu.",
            "intent": "multi_room_control",
            "rooms": ["bedroom", "kitchen"],
            "actions": [
              {"room": "bedroom", "action": "turn_off"},
              {"room": "kitchen", "action": "adjust_brightness", "brightness": "low"}
            ]
        }

        Example 2:
        {
            "id": 1,
            "command": "Sutney kotha lights off and kitchen lights dim garna sakincha?",
            "intent": "multi_room_control",
            "rooms": ["bedroom", "kitchen"],
            "actions": [
              {"room": "bedroom", "action": "turn_off"},
              {"room": "kitchen", "action": "adjust_brightness", "brightness": "low"}
            ]
        }

        Example 3:
        {
            "id": 1,
            "command": "Living room lights halka matra ujyalo garnu. Bedroom lights chahi off garnu.",
            "intent": "multi_room_control",
            "rooms": ["living room", "bedroom"],
            "actions": [
              {"room": "living room", "action": "adjust_brightness", "brightness" : "low"},
              {"room": "bedroom", "action": "turn_off"}
            ]
        }

        Ensure the commands meet these criteria:
        1. 50% of commands involve multiple rooms.
        2. 20% of commands should be ambiguous or vague.
        3. Include commands with varying tone (polite, conversational, formal, informal).
        4. Commands must include diverse intents (turn_on, turn_off, adjust_brightness, multi_room_control).
        5. Examples of vague commands: "Sabai lights bright garidinus." or "Sutney kotha dim garera baithak ujyalo rakhnus."
        6. Alternatively use the Nepali and English words for the rooms in command property but always use English words for rooms in rooms and actions property.

        Test Cases to Cover:
        - Ambiguity: Vague commands without specific rooms or brightness.
        - Multi-room: Commands involving multiple rooms and actions.
        - Complex Intents: Combination of actions (turn off, dim, brighten).
        
        ### Example Commands:
        1. 'Bedroom lights off garera bhancha kotha maa lights dim garidinu.'
        2. 'Living room lights thap bright banaunus.'
        3. 'Sabai room ka lights dim garidinus.'
        4. 'Turn off living room lights and change bedroom lights to red.'
        5. 'Bahira ko lights dim garera kitchen ko lights bright garidinus.'
        6. 'Hallway ko lights chahakilo banaunus.
        
        Write the output as JSON objects following the example format above.'''


In [11]:
def builder(system, user):
    return [{"role":"system", "content":system},
            {"role":"user", "content":user}
           ]       

In [12]:
# combining all the steps
def generate_dataset(system, user):
    try:
        response = openai.chat.completions.create(model = "gpt-4o",
                                                  messages = builder(system, user),
                                                  # temperature = 0.8, 
                                                  # max_tokens = 2000
                                                  )
        # dataset = json.loads(response.choices[0].message.content)
        dataset = (response.choices[0].message.content)
        return dataset
    except Exception as e:
        print(f"Error generating dataset: {str(e)}")
        return []

In [13]:
# function to save the dataset as json      
def save_dataset(dataset: List[Dict], filename: str):
    """Save the generated dataset to a JSON file"""
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(dataset, f, ensure_ascii=False, indent=4)

In [14]:
if __name__ == "__main__":
    dataset = generate_dataset(system, user)
    
    if dataset:
        # Save the dataset
        dataset1 = dataset[7:]
        dataset1 = dataset1[:-3]
        dataset1 = json.loads(dataset1)
        save_dataset(dataset1, "light_automation_dataset.json")
        print(f"Successfully generated and saved {len(dataset1)} samples to light_automation_dataset.json")
    else:
        print("Failed to generate dataset")

Successfully generated and saved 10 samples to light_automation_dataset.json
